# Using EDSL to sense check data
This notebook provides example code for sense checking survey data using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and market research with AI agents and large language models. 

## Contents
Using a set of responses to a survey about online marketplaces as an example, we demonstrate EDSL methods for: 

1. Evaluating survey questions (e.g., for clarity and improvements)
2. Analyzing each respondent's set of answers (e.g., to summarize or identify sentiment, themes, etc.)
3. Reviewing each answer individually (e.g., to evaluate its relevance or usefulness)

## Coop
We also show how to post EDSL questions, surveys, results and notebooks (like this one) to [Coop: an integrated platform for creating and sharing LLM-based research](https://www.expectedparrot.com/explore). 

## How EDSL works
EDSL is a flexible library that can be used to perform a broad variety of research tasks. A typical workflow consists of the following steps:

* Construct questions  
* Add data to the questions (e.g., for data labeling tasks)
* Use an AI agent to answer the questions
* Select a language model to generate the answers
* Analyze results in a formatted dataset

## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library.
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) OR store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) provides examples of EDSL basic components. 

## Example data
Our example data is a CSV consisting of several questions and a few rows of responses.
Here we store it at the Coop and then re-import it.

To post an object:

    from edsl import FileStore
    
    fs = FileStore("marketplace_survey_results.csv")
    fs.push(description = "mock marketplace survey results")

This returns details of the object we can use to retrieve it:

    {'description': 'mock marketplace survey results',
     'object_type': 'scenario',
     'url': 'https://www.expectedparrot.com/content/7e8d0aa8-dfc6-42e5-90a6-4b428d0d9b4c',
     'uuid': '7e8d0aa8-dfc6-42e5-90a6-4b428d0d9b4c',
     'version': '0.1.47.dev1',
     'visibility': 'unlisted'}

In [1]:
from edsl import FileStore

In [2]:
csv_file = FileStore.pull("7e8d0aa8-dfc6-42e5-90a6-4b428d0d9b4c")

## Creating questions about the data
There are many questions we might want to ask about the data, such as:

* Does this survey question have any logical or syntactical problems? {{ *question* }}
* What is the overall sentiment of this respondent's answers? {{ *responses* }}
* Is this answer responsive to the question that was asked? {{ *question* }} {{ *answer* }}

## Question types
EDSL comes with many common question types that we can select from based on the form of the response that we want to get back from the model: multiple choice, checkbox, linear scale, free text, etc. [Learn more about EDSL question types](https://docs.expectedparrot.com/en/latest/questions.html).

Here we construct `Question` objects for the questions that we want to ask about the data, using `{{ placeholders }}` for the information that we will add to the questions in the steps that follow:

In [3]:
from edsl import QuestionFreeText, QuestionMultipleChoice, QuestionYesNo

In [4]:
q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = "Describe any logical or syntactical problems in the following survey question: {{ scenario.question }}"
)

In [5]:
q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the overall sentiment of this respondent's survey answers? {{ scenario.responses }}",
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

In [6]:
q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = "Is this answer responsive to the question that was asked? Question: {{ scenario.question }} Answer: {{ scenario.answer }}"
)

## Adding survey data to the questions
Next we'll add our data to our questions. This can be done efficiently by creating a `ScenarioList` representing the data. The individual `Scenario` objects in the list can be constructed in a variety of ways depending on the information that we want to include in a particular question.

We start by calling the `from_csv()` method to create a `ScenarioList` for the data in its original form. We can see that this generates a `Scenario` dictionary for each respondent's set of answers with key/value pairs for the individual questions and answers: 

In [7]:
from edsl import ScenarioList

In [8]:
sl = ScenarioList.from_source("csv", csv_file.to_tempfile()) # replace with CSV file name if importing a local file
sl

,Respondent ID,What do you like most about using our online marketplace?,What is one feature you would like to see added to improve your shopping experience?,Can you describe a recent experience where you were dissatisfied with our service?,How do you feel about the current product search and filtering options?,Is there anything else you would like to share about your experience with us?
0,101,The wide variety of products and the ease of use.,It would be great to have a personalized recommendation system based on my browsing history.,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.",The search and filtering options are intuitive and work well for me.,"No, keep up the great work!"
1,102,I enjoy the simplicity of the interface.,A feature that helps compare similar products side by side would be useful.,No complaints here.,I find the product search to be pretty effective.,I think the sky is a beautiful shade of purple today.
2,103,The platform is user-friendly and offers a vast selection of products.,Would love to see an option to save and compare different products.,"My delivery was late by a few days, which was frustrating.",It’s okay.,No.


## Evaluating the questions
For our first question we want to create a `Scenario` for each survey question:

In [9]:
from edsl import QuestionFreeText, Survey

q_logic = QuestionFreeText(
    question_name = "logic",
    question_text = "Describe any logical or syntactical problems in the following survey question: {{ scenario.question }}"
)

q_improved = QuestionFreeText(
    question_name = "improved",
    question_text = "Please draft an improved version of the survey question. Return only the revised question text."
)

survey = Survey([q_logic, q_improved]).add_targeted_memory(q_improved, q_logic)

The survey questions are the `parameters` of the `ScenarioList` created above:

In [10]:
questions = list(sl.parameters)
questions

['Is there anything else you would like to share about your experience with us?',
 'How do you feel about the current product search and filtering options?',
 'Respondent ID',
 'What do you like most about using our online marketplace?',
 'What is one feature you would like to see added to improve your shopping experience?',
 'Can you describe a recent experience where you were dissatisfied with our service?']

We can pass them to the `from_list()` method to create a new `ScenarioList`, specifying that the key for each `Scenario` will be `question` in order to match the parameter of our logic question:

In [11]:
sl_questions = ScenarioList.from_list("question", questions)
sl_questions

,question
0,Is there anything else you would like to share about your experience with us?
1,How do you feel about the current product search and filtering options?
2,Respondent ID
3,What do you like most about using our online marketplace?
4,What is one feature you would like to see added to improve your shopping experience?
5,Can you describe a recent experience where you were dissatisfied with our service?


We select a model to use, and then add the scenarios to the survey when we run it:

In [12]:
from edsl import Model

m = Model("gemini-1.5-flash")

In [13]:
results = survey.by(sl_questions).by(m).run()

This generates a dataset of `Results` that we can access with built-in methods for analysis:

In [14]:
results.select("question", "logic", "improved")

,scenario.question,answer.logic,answer.improved
0,Is there anything else you would like to share about your experience with us?,"The question ""Is there anything else you would like to share about your experience with us?"" has a few potential problems, mostly related to its openness and lack of guidance: * **Too open-ended:** It's so broad that respondents might struggle to know where to begin. They might share irrelevant information, provide overly lengthy responses making analysis difficult, or simply not know what to say. * **Lack of specificity:** ""Your experience with us"" is vague. Was it a single interaction, a product, a service, a period of time? The more specific the question, the more useful the responses. * **Potential for bias:** The phrasing is slightly leading. The word ""else"" implies there's already been *something* shared, potentially influencing the respondent to think they *must* add something further, even if they have nothing more to say. * **No guidance on format:** Should respondents write a paragraph, a list, or just a single sentence? Providing some guidance on the desired response length or format would improve the quality and consistency of the data. In short, while seemingly innocuous, the question lacks the structure and guidance needed for effective survey design. It could be improved significantly by making it more specific and providing clearer instructions.","Thinking about your recent experience with us, is there anything specific you would like to share? Please limit your response to one or two sentences."
1,How do you feel about the current product search and filtering options?,"The question ""How do you feel about the current product search and filtering options?"" has a few potential problems: * **Too broad and subjective:** ""How do you feel"" is very open-ended. It doesn't provide a framework for the respondent to answer concisely or consistently. One person might describe a general feeling (""Frustrated""), while another might launch into a detailed account of specific issues. This makes the data difficult to analyze and compare. * **Ambiguous scope:** ""Current product search and filtering options"" is vague. Does it refer to all products on the site? A specific product category? The respondent might interpret this differently than the survey creator intended. More specificity is needed. * **Lacks specific prompts for feedback:** The question doesn't guide the respondent towards specific aspects of the search and filtering. It would be better to break this down into multiple questions focusing on individual aspects, like ease of use, accuracy of results, speed, clarity of filters, etc. To improve the question, it could be broken down into several more specific questions, such as: * ""How easy was it to find the products you were looking for?"" (with a rating scale) * ""How accurate were the search results?"" (with a rating scale) * ""How would you rate the speed of the search?"" (with a rating scale) * ""Were the filtering options clear and easy to understand?"" (yes/no or rating scale) * ""What, if anything, could be improved about the search and filtering options?"" (open-ended text box) This approach provides more structured data, allowing for more meaningful analysis and actionable insights.",How easy was it to find the products you were looking for?
2,Respondent ID,"""Respondent ID"" is not a survey *question*; it's a field for identifying the respondent's data. It's not something you *ask* the respondent, but rather something you assign or they provide (e.g., a study number, an email address used for identification). Therefore, the problem isn't logical or syntactical within the phrase itself, but rather its inappropriate use as a survey *question*. It's a data entry field, not a question prompting a response from the participant.",What is your Respondent ID?
3,What do you like most about using our online marketplace?,"The main problem with the question ""What do you like mos

[Learn more about working with results](https://docs.expectedparrot.com/en/latest/results.html).

## Evaluating respondents' collective answers
Next we can create a `ScenarioList` for each respondent's answers to use with our question about sentiment:

In [15]:
sl

,Respondent ID,What do you like most about using our online marketplace?,What is one feature you would like to see added to improve your shopping experience?,Can you describe a recent experience where you were dissatisfied with our service?,How do you feel about the current product search and filtering options?,Is there anything else you would like to share about your experience with us?
0,101,The wide variety of products and the ease of use.,It would be great to have a personalized recommendation system based on my browsing history.,"I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.",The search and filtering options are intuitive and work well for me.,"No, keep up the great work!"
1,102,I enjoy the simplicity of the interface.,A feature that helps compare similar products side by side would be useful.,No complaints here.,I find the product search to be pretty effective.,I think the sky is a beautiful shade of purple today.
2,103,The platform is user-friendly and offers a vast selection of products.,Would love to see an option to save and compare different products.,"My delivery was late by a few days, which was frustrating.",It’s okay.,No.


In [17]:
sl_responses = ScenarioList.from_list("responses", [str(x) for x in sl])
sl_responses

,responses
0,"Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'})"
1,"Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'})"
2,"Scenario({'Respondent ID': '103', 'What do you like most about using our online marketplace?': 'The platform is user-friendly and offers a vast selection of products.', 'What is one feature you would like to see added to improve your shopping experience?': 'Would love to see an option to save and compare different products.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'My delivery was late by a few days, which was frustrating.', 'How do you feel about the current product search and filtering options?': 'It’s okay.', 'Is there anything else you would like to share about your experience with us?': 'No.'})"


Next we add these scenarios to our sentiment question (and any others we want to add) and run it:

In [18]:
from edsl import QuestionMultipleChoice, QuestionLinearScale, Survey

q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the overall sentiment of this respondent's survey answers? {{ scenario.responses }}",
    question_options = ["Very unsatisfied", "Somewhat unsatisfied", "Somewhat satisfied", "Very satisfied"]
)

q_recommend = QuestionLinearScale(
    question_name = "recommend",
    question_text = "On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? {{ scenario.responses }}",
    question_options = [1, 2, 3, 4, 5],
    option_labels = {1:"Not at all likely", 5:"Very likely"}
)

survey = Survey([q_sentiment, q_recommend])

In [19]:
jobs = results = survey.by(sl_responses).by(m)
jobs.prompts()

,user_prompt,system_prompt,interview_index,question_name,scenario_index,agent_index,model,estimated_cost,cache_keys
0,"What is the overall sentiment of this respondent's survey answers? Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}) Very unsatisfied Somewhat unsatisfied Somewhat satisfied Very satisfied Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,sentiment,0,0,gemini-1.5-flash,0.000086,['a9f07abede664248b812151e58689afc']
1,"On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? Scenario({'Respondent ID': '101', 'What do you like most about using our online marketplace?': 'The wide variety of products and the ease of use.', 'What is one feature you would like to see added to improve your shopping experience?': 'It would be great to have a personalized recommendation system based on my browsing history.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'I was disappointed when an item I ordered arrived damaged, but customer service quickly resolved it.', 'How do you feel about the current product search and filtering options?': 'The search and filtering options are intuitive and work well for me.', 'Is there anything else you would like to share about your experience with us?': 'No, keep up the great work!'}) 1 : Not at all likely 2 : 3 : 4 : 5 : Very likely Only 1 option may be selected. Respond only with the code corresponding to one of the options. E.g., ""1"" or ""5"" by itself. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,0,recommend,0,0,gemini-1.5-flash,0.000088,['b2d11e87d3c0d3e5cd0862481448dd5d']
2,"What is the overall sentiment of this respondent's survey answers? Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feature that helps compare similar products side by side would be useful.', 'Can you describe a recent experience where you were dissatisfied with our service?': 'No complaints here.', 'How do you feel about the current product search and filtering options?': 'I find the product search to be pretty effective.', 'Is there anything else you would like to share about your experience with us?': 'I think the sky is a beautiful shade of purple today.'}) Very unsatisfied Somewhat unsatisfied Somewhat satisfied Very satisfied Only 1 option may be selected. Respond only with a string corresponding to one of the options. After the answer, you can put a comment explaining why you chose that option on the next line.",nan,1,sentiment,1,0,gemini-1.5-flash,0.000078,['30fd4ffc4ed0e285a662681d3b07bfca']
3,"On a scale from 1 to 5, how likely do you think this respondent is to recommend the company to a friend? Scenario({'Respondent ID': '102', 'What do you like most about using our online marketplace?': 'I enjoy the simplicity of the interface.', 'What is one feature you would like to see added to improve your shopping experience?': 'A feat

In [20]:
results = survey.by(sl_responses).by(m).run()

In [ ]:
results.select("responses", "sentiment", "recommend")

## Evaluating individual answers
Next we create a `ScenarioList` for each individual question and answer to use with our question about the responsiveness of each answer. We can use the `unpivot()` method to expand the scenarios by desired identifiers (e.g., respondent ID):

In [ ]:
sl_qa = sl.unpivot(id_vars = ["Respondent ID"])
sl_qa

We can call the `rename()` method to rename the keys as desired to match our question parameters syntax:

In [ ]:
sl_qa = sl_qa.rename({"Respondent ID": "id", "variable": "question", "value": "answer"})
sl_qa

In [21]:
from edsl import QuestionYesNo

q_responsive = QuestionYesNo(
    question_name = "responsive",
    question_text = "Is this answer responsive to the question that was asked? Question: {{ scenario.question }} Answer: {{ scenario.answer }}"
)

In [ ]:
results = q_responsive.by(sl_qa).by(m).run()

In [ ]:
(
    results
    .filter("responsive == 'No'")
    .select("id", "question", "answer")
)

## Uploading content to Coop
[Coop](https://www.expectedparrot.com/content/content/explore) is a new platform for creating, storing and sharing LLM-based research. It is fully integrated with EDSL, and a convenient place to post and access surveys, agents, results and notebooks. [Learn more about using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the contents of this notebook:

In [26]:
from edsl import Notebook

nb = Notebook(path = "scenariolist_unpivot.ipynb")

if refresh := False:
    nb.push(
        description = "ScenarioList methods for sense checking survey data", 
        alias = "scenariolist-sense-checking-survey-notebook",
        visibility = "public"
    )
if patch := False:
    nb.patch('aad6b4f6-f7f3-4d67-9a79-8ce105f289fd', value = nb)